# Introduction
This notebook demonstrates the integration of roboquant with Interactive Brokers (IBKR). 
<div class="alert alert-block alert-success">
<img src="https://advisorhub.com/wp-content/uploads/2019/04/IB-Logo-blk-text.png" width=400 />
</div>

> Disclaimer: roboquant is not affiliated with Interactive Brokers Group, Inc.’s.


There are currently 3 types of integration available:

1. Use historic data from IBKR for back testing 
2. Use live data from IBKR for live testing 
2. Use the IBKR trading functionality (WIP) for paper trading

<div class="alert alert-block alert-warning">
Make sure you always select paper trading account in your IB Gateway or Trader Workstation, since roboquant is still very much beta. If you didn't yet setup a paper trading account, click this <a href="https://ibkr.info/node/663">article</a>  for some more info
</div>


We'll run a very simple strategy since the focus is IBKR integration for this notebook. But there are other notebooks available that focus on how to develop custom strategies and other aspect of roboquant like charting.


# Installation

The setup and integration with IBKR is a more complicated than with other brokers. The two extra steps that are required:

1.You cannot connect directly to an IBKR server. You will always need to run either ```IB Gateway``` or ```Trader Workstation```, and roboquant will connect to that instance. ```IB Gateway``` is the better option of the two, since it provides nice insights into the called API's and is less heavy weight. If you don't have it yet installed, you can download IB Gateway from [here](https://www.interactivebrokers.co.uk/en/index.php?f=16454)

> Because you need a running IB Gateway that is accessible, you won't be able to successfully run this notebook on public infrastructure like MyBinder.org or Datalore by JetBrains.

2. The TWS-API client cannot be packaged together with roboquant due to license constraints. So you'll have to download this file yourself, and then make it available to roboquant by uploading it to the tutorials directory.  

    - Download the [TWS API client](https://interactivebrokers.github.io/downloads/twsapi_macunix.976.01.zip) from the IBKR website.
    - Unzip the downloaded file and locate the file called ```TwsApi.jar```. It can be found at following path ```IBJts/source/JavaClient/TwsApi.jar``` inside the extracted folder.
    - Upload this single jar file to the notebook directory.


After that, you can run the next cell and if no errors you are good to go.

In [ ]:
%use roboquant(0.9-SNAPSHOT)
@file:DependsOn("TwsApi.jar")
@file:DependsOn("org.roboquant:roboquant-ibkr:0.9-SNAPSHOT")
import org.roboquant.ibkr.*
Welcome()

# Back Testing

The first part shows how to retrieve historic prices and run a back test. In order to keep is simple we retrieve a single asset, but you can retrieve many assets and they will all be available during the back testing. 
Because IBKR offers data for many different assets in different markets, you need to specify what specific assets you are interested in. A simple symbol name won't be sufficient. For stock type of assets, at least supply the Symbol, Currency and possible the Exchange.


> Typically a paid subscription is required to receive market data from IBKR. Also IBKR has strict limitations on how often you can make an API call. So if you require high frequency market data for running your strategy, IBKR market data subscriptions might not be the best fit.

In [ ]:
// Uncomment an asset that you have a market data subscription for
val symbols = listOf("TSLA", "MSFT", "GOOGL", "JPM")
val assets = symbols.map { Asset(it) }.toTypedArray()

In [ ]:
val feed = IBKRHistoricFeed()
val endDate = Instant.parse("2022-01-01T00:00:00Z")
feed.retrieve(*assets, endDate = endDate)

In [ ]:
PriceBarChart(feed, assets.first())

In [ ]:
val strategy = EMACrossover(3, 5)
val roboquant =  Roboquant(strategy, AccountSummary())

In [ ]:
roboquant.run(feed)

In [ ]:
val account = roboquant.broker.account
account.summary()

In [ ]:
val metricResults = roboquant.logger.getMetric("account.equity")
MetricChart(metricResults)

In [ ]:
PriceChart(feed, assets.first(), account.trades)

# Live Testing
Now we create an instance of the IBKRFeed and subscribe to the same asset. The setup is very much identical as back testing, the only difference is that now we use a live data feed instead of a historic data feed.
Of course, outside trading hours there won't be price data and no signals or orders will be generated.  

In [ ]:
val feed = IBKRLiveFeed()
feed.subscribe(*assets)

In [ ]:
val strategy = EMACrossover(3, 5)
val eventRecorder = EventRecorder()
val roboquant =  Roboquant(strategy, AccountSummary(), eventRecorder)

We have all the components assembled that we need to start the test. All that remains, is to start the run and evaluate the strategy against the feed. 
We'll run it for 30 minutes, but you can change this. But remember the EMA Crossover is only going to evaluate after at least three observations.

If the time that is displayed in the progress bar looks off by a few hours, please realize that roboquant uses a timezone independent representation for all internal time processing (same as UTC).

In [ ]:
val timeframe = Timeframe.next(30.minutes)
roboquant.run(feed, timeframe)
feed.disconnect()

## Results
The run has completed, lets see a few result. Of course if the run was outside trading hours without any price action, there will not be much to see.

In [ ]:
val account = roboquant.broker.account
account.summary()

In [ ]:
account.portfolio.summary()

In [ ]:
val logger = roboquant.logger
logger.summary()

## Charts
Also lets plot two charts. See also the visualization notebook for examples of how to use charts to display results

In [ ]:
logger.metricNames.summary()

In [ ]:
val accountValue = logger.getMetric("account.equity")
MetricChart(accountValue, useTime = false)

In [ ]:
TradeChart(account)

# Paper Trading
And just as it was the case in the previous sections, most live feeds will only generate data during trading hours. So if you run these code cells outside trading hours, you won't see signals and orders being generated (depending of course on the used strategy).

<div class="alert alert-block alert-warning">
The integration with the trading API is still very much work in progress, so only use this with a paper trading account and even then it can lead to unwatned updates to your paper trading account.
</div>

In [ ]:
val broker = IBKRBroker(enableOrders = true)
Thread.sleep(5000)

In [ ]:
broker.account.fullSummary()

## Place an order directly
Besides using strategies and policies to generate orders, you can also call the `broker.place` method directly to place an order. Again, use this only with paper trading accounts. 

In [ ]:
val asset = broker.account.portfolio.assets.findBySymbols("AAPL").first()
val order = MarketOrder(asset, 2.0)
broker.place(listOf(order), Event.empty())

// Should show up as an open order initially
broker.account.fullSummary()


// Now should show up as an extra closed order (if placed during trading hours)
// Also you might hear TWS say "Order Filled"
Thread.sleep(5000)
broker.account.fullSummary()

In [ ]:
broker.account.trades

In [ ]:
broker.account.trades.first().fee

In [ ]:
broker.account.orders.summary()

In [ ]:
val strategy = EMACrossover(3, 5)
val roboquant = Roboquant(strategy, AccountSummary(), broker = broker)

The setup of the feed is exactly the same as before. There is no difference in the feed if you use it for live testing with the builtin Simulated Broker or paper- and live-trading. Alos this time we'll run it for 30 minutes.

In [ ]:
val timeframe = Timeframe.next(30.minutes)
roboquant.run(feed, timeframe)

In [ ]:
broker.account.portfolio.summary()

In [ ]:
roboquant.logger.summary()

In [ ]:
feed.disconnect()

In [ ]:
broker.disconnect()